In [8]:
from pkg.io import savefig
import os
import umap
import pymaid
import logging
import pandas as pd
import numpy as np
import networkx as nx
from itertools import chain, combinations
from upsetplot import plot
from matplotlib import pyplot as plt
from networkx import from_numpy_array, number_of_nodes, number_of_edges, density
from graspologic.embed import AdjacencySpectralEmbed
from graspologic.layouts import layout_tsne, layout_umap
from graspologic.plot.plot import networkplot
from graspologic.utils import is_fully_connected, largest_connected_component, is_symmetric, symmetrize
from graspologic.simulations import sbm

In [2]:
rm = pymaid.CatmaidInstance(
    server="https://catmaid.jekelylab.ex.ac.uk/#",
    project_id=11,
    api_token=None,
    http_user=None,
    http_password=None,
)
logging.getLogger("pymaid").setLevel(logging.WARNING)
pymaid.clear_cache()

INFO  : Global CATMAID instance set. Caching is ON. (pymaid)


In [3]:
skids_orig = pymaid.get_skids_by_annotation("connectome")

In [4]:
#original dataframe adjacency matrix and skids
adj_df_orig = pymaid.adjacency_matrix(skids_orig)

In [5]:
#numpy representation of original graph
adj_numpy_orig = adj_df_orig.to_numpy()

#numpy largest connected component
adj_numpy_lcc, new_inds = largest_connected_component(adj_numpy_orig, return_inds=True)
#get skids of nodes in lcc
skids_lcc = adj_df_orig.index[new_inds]

#new pandas adj matrix
#adj_df_lcc = adj_df_orig.iloc[inds].iloc[:, inds]

In [7]:
ase = AdjacencySpectralEmbed()
X = ase.fit_transform(adj_numpy_lcc)
X

(array([[ 8.72793551e-03,  2.07335751e-02,  1.62184287e-02, ...,
          1.56657758e-02,  2.73920062e-02,  3.07069518e-01],
        [ 1.57499245e-13,  8.66640011e-11,  4.69619500e-11, ...,
         -1.69834111e-10, -8.53700077e-11,  1.44810956e-09],
        [ 5.70111308e-04,  1.38313472e-04,  1.49469404e-04, ...,
          3.16049849e-04,  1.92535398e-04,  1.56560679e-03],
        ...,
        [ 9.30381684e-06,  1.27611690e-04,  1.47724863e-04, ...,
          5.44113182e-04, -1.18722980e-05,  3.12188848e-04],
        [ 1.89692454e-07,  8.53342048e-07,  9.21744189e-07, ...,
          7.37714724e-06,  1.06440112e-05,  1.23456728e-05],
        [ 1.64222531e-11,  1.93829219e-10,  2.24763285e-10, ...,
          2.77019829e-10,  1.09779855e-09,  5.41345756e-09]]),
 array([[ 1.42257953e-04,  3.05057290e-04,  3.37796286e-04, ...,
          2.84901940e-04,  1.37650983e-03,  6.02999134e-03],
        [ 5.09168788e-07,  1.08809429e-05,  4.05546049e-06, ...,
         -3.66457148e-06,  2.98847376e

In [11]:
points = umap.UMAP(
        min_dist=0.75, n_neighbors=25, random_state=None
    ).fit_transform(X)
points

ValueError: Found array with dim 3. Estimator expected <= 2.